In [7]:
import pandas as pd
import os # Added os module to help with path creation

def load_and_clean_data(btc_file_path, gold_file_path, output_directory, start_date_str, end_date_str):
    """
    Loads, cleans, merges BTC and Gold time series data, filters by date, and saves the result.

    Args:
        btc_file_path (str): File path for the BTC-USD CSV data.
        gold_file_path (str): File path for the new Gold CSV data.
        output_directory (str): Directory where the merged CSV will be saved.
        start_date_str (str): Start date for filtering (YYYY-MM-DD).
        end_date_str (str): End date for filtering (YYYY-MM-DD).


    Returns:
        pandas.DataFrame: A merged and filtered DataFrame with hourly data, or None if an error occurs.
    """
    try:
        # --- Load the first dataset (BTC-USD) ---
        print(f"Loading BTC data from: {btc_file_path}")
        df1 = pd.read_csv(btc_file_path)
        print("BTC data loaded successfully. First 5 rows:")
        print(df1.head())

        # Convert 'datetime' column to datetime objects and set as index
        df1['datetime'] = pd.to_datetime(df1['datetime'])
        df1.set_index('datetime', inplace=True)
        df1.sort_index(inplace=True)
        print("\nBTC data after setting datetime index (first 5 rows):")
        print(df1.head())
        print(f"BTC data shape: {df1.shape}")


        # --- Load the second dataset (New Gold Data) ---
        print(f"\nLoading Gold data from: {gold_file_path}")
        # The new gold data uses ';' as a separator and has a specific date format.
        df2 = pd.read_csv(gold_file_path, sep=';')
        print("Gold data loaded successfully. First 5 rows:")
        print(df2.head())
        print("\nGold data info:")
        df2.info()


        # Rename the date column for consistency, e.g., from 'Date' to 'time'
        # Assuming the date column in the new gold CSV is named 'Date'
        if 'Date' not in df2.columns:
            # Attempt to find a date-like column if 'Date' is not present
            # This is a simple check; more robust detection might be needed for varied files
            date_col_candidate = None
            for col in df2.columns:
                if 'date' in col.lower() or 'time' in col.lower():
                    date_col_candidate = col
                    break
            if date_col_candidate:
                print(f"Warning: 'Date' column not found. Using '{date_col_candidate}' as date column.")
                df2.rename(columns={date_col_candidate: 'time_gold'}, inplace=True)
            else:
                raise ValueError("Date column ('Date' or similar) not found in Gold CSV. Please check the CSV header.")
        else:
            df2.rename(columns={'Date': 'time_gold'}, inplace=True)


        # Convert 'time_gold' column to datetime objects
        # The format is 'YYYY.MM.DD HH:MM'
        df2['time_gold'] = pd.to_datetime(df2['time_gold'], format='%Y.%m.%d %H:%M')
        df2.set_index('time_gold', inplace=True)
        df2.sort_index(inplace=True)
        print("\nGold data after parsing date and setting index (first 5 rows):")
        print(df2.head())
        print(f"Gold data shape: {df2.shape}")

        # Rename columns in df2 to avoid conflicts (e.g., 'Open' to 'gold_Open')
        # The columns in the new gold data are 'Open', 'High', 'Low', 'Close', 'Volume'
        df2_columns_renamed = {col: f"gold_{col}" for col in df2.columns}
        df2 = df2.rename(columns=df2_columns_renamed)
        print("\nGold data after renaming columns (first 5 rows):")
        print(df2.head())

        # The new gold data is already hourly.
        # If there were any irregularities or if we wanted to ensure perfect hourly frequency alignment,
        # resampling could be used, but it might not be strictly necessary if data is clean.
        # For safety and consistency with previous logic, we can resample.
        # This will also ensure the DatetimeIndex has a frequency set.
        print("\nEnsuring Gold data has hourly frequency (resampling if needed)...")
        df2_resampled = df2.resample('H').ffill() # Forward fill any potential gaps
        print("Gold data after ensuring hourly frequency (first 5 rows):")
        print(df2_resampled.head())
        print(f"Gold data (resampled) shape: {df2_resampled.shape}")


        # --- Merge the two DataFrames ---
        print("\nMerging the two DataFrames (BTC and Gold)...")
        # Using a 'left' merge to keep all timestamps from df1 (BTC data)
        # and match df2_resampled (Gold data).
        merged_df = pd.merge(df1, df2_resampled, left_index=True, right_index=True, how='left')
        print("Data merged successfully. First 5 rows of merged data:")
        print(merged_df.head())
        print(f"Merged_df shape before date filtering: {merged_df.shape}")


        # --- Data Cleaning: Handling Missing Values after Merge ---
        print(f"\nMissing values before specific handling in merged_df:\n{merged_df.isnull().sum()}")
        # Forward fill missing values that might have resulted from the left merge for gold columns
        gold_cols = [col for col in merged_df.columns if col.startswith('gold_')]
        if gold_cols:
            merged_df[gold_cols] = merged_df[gold_cols].ffill()
            # Optionally, backward fill any remaining NaNs at the beginning
            merged_df[gold_cols] = merged_df[gold_cols].bfill()
            print("\nMissing values after forward/backward filling Gold columns:")
            print(merged_df[gold_cols].isnull().sum())


        # --- Filter by Date Range ---
        print(f"\nFiltering merged data from {start_date_str} to {end_date_str} (inclusive)...")
        # Convert string dates to datetime objects for filtering
        start_datetime = pd.to_datetime(start_date_str + " 00:00:00") # Inclusive start of the day
        end_datetime = pd.to_datetime(end_date_str + " 23:59:59")   # Inclusive end of the day

        # Ensure the index is a DatetimeIndex (should be already)
        merged_df.index = pd.to_datetime(merged_df.index)

        original_rows_before_filter = len(merged_df)
        merged_df_filtered = merged_df[(merged_df.index >= start_datetime) & (merged_df.index <= end_datetime)]
        print(f"Number of rows before date filter: {original_rows_before_filter}")
        print(f"Number of rows after date filter: {len(merged_df_filtered)}")

        if merged_df_filtered.empty:
            print(f"Warning: No data found in the specified date range: {start_date_str} to {end_date_str}.")
            print("Please check the date range and the date coverage of your input files.")
        else:
            print("\nFiltered merged data (first 5 rows):")
            print(merged_df_filtered.head())
            print("\nFiltered merged data (last 5 rows):")
            print(merged_df_filtered.tail())

        print("\nFinal merged and filtered data info:")
        merged_df_filtered.info()


        # --- Save the merged and filtered DataFrame ---
        output_filename = "merged_btc_gold_hourly_data_filtered.csv"
        output_path = os.path.join(output_directory, output_filename)

        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
            print(f"\nCreated output directory: {output_directory}")

        merged_df_filtered.to_csv(output_path)
        print(f"\nMerged and filtered data saved to: {output_path}")

        return merged_df_filtered

    except FileNotFoundError as e:
        print(f"Error: File not found. Please check the file paths. Details: {e}")
        return None
    except ValueError as e: # Catch specific errors like date parsing or column not found
        print(f"Error during data processing: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# --- Configuration: File Paths and Date Range ---
# Path to the BTC-USD data
btc_file_path = r"C:\Users\User\Downloads\KPZHackathon\final_merged_data_2.csv"

# Path to the NEW Gold data
new_gold_file_path = r"C:\Users\User\Downloads\XAU_1h_data.csv"

# Directory to save the output file
output_dir = r"C:\Users\User\Downloads\KPZHackathon"

# Desired date range for the final output
filter_start_date = "2020-04-09"
filter_end_date = "2025-01-31" # Note: Data will only go up to the latest available in your inputs.

# Call the function to load, merge, filter, and save data
final_dataframe = load_and_clean_data(btc_file_path, new_gold_file_path, output_dir, filter_start_date, filter_end_date)

if final_dataframe is not None and not final_dataframe.empty:
    print("\n--- Data Merging, Filtering, and Saving Process Completed Successfully ---")
    print(f"Total rows in the final DataFrame: {len(final_dataframe)}")
    print(f"Columns in the final DataFrame: {final_dataframe.columns.tolist()}")
    print(f"Date range in final DataFrame: {final_dataframe.index.min()} to {final_dataframe.index.max()}")
elif final_dataframe is not None and final_dataframe.empty:
    print("\n--- Data Merging and Saving Process Completed, but the DataFrame is EMPTY after filtering. ---")
    print("This means no data matched your specified date range or other criteria.")
else:
    print("\n--- Data Merging, Filtering, and Saving Process Failed ---")



Loading BTC data from: C:\Users\User\Downloads\KPZHackathon\final_merged_data_2.csv
BTC data loaded successfully. First 5 rows:
              datetime  transactions_count_flow_1  current_regime  \
0  2020-04-09 00:00:00                        401               5   
1  2020-04-09 01:00:00                        558               5   
2  2020-04-09 02:00:00                        633               5   
3  2020-04-09 03:00:00                        488               5   
4  2020-04-09 04:00:00                        709               5   

   predicted_next_regime  next_regime_probability     start_time  inflow_mean  \
0                      5                  0.46246  1586390400000     0.800575   
1                      5                  0.46246  1586394000000     0.532598   
2                      5                  0.46246  1586397600000     0.531611   
3                      5                  0.46246  1586401200000     0.688415   
4                      5                  0.46246  1

C:\Users\User\AppData\Local\Temp\ipykernel_20304\1066436707.py:86: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2_resampled = df2.resample('H').ffill() # Forward fill any potential gaps


Number of rows before date filter: 43225
Number of rows after date filter: 42216

Filtered merged data (first 5 rows):
                     transactions_count_flow_1  current_regime  \
datetime                                                         
2020-04-09 00:00:00                        401               5   
2020-04-09 01:00:00                        558               5   
2020-04-09 02:00:00                        633               5   
2020-04-09 03:00:00                        488               5   
2020-04-09 04:00:00                        709               5   

                     predicted_next_regime  next_regime_probability  \
datetime                                                              
2020-04-09 00:00:00                      5                  0.46246   
2020-04-09 01:00:00                      5                  0.46246   
2020-04-09 02:00:00                      5                  0.46246   
2020-04-09 03:00:00                      5                  0.4